In [1]:
# ============================================================
# Notebook setup: run this before everything
# ============================================================

%load_ext autoreload
%autoreload 2

# Control figure size
interactive_figures = False
if interactive_figures:
    # Normal behavior
    %matplotlib widget
    figsize=(9, 3)
else:
    # PDF export behavior
    figsize=(14, 5)

#from matplotlib import pyplot as plt
from util import util
import igraph as ig
import numpy as np

# ============================================================
# Repeat relevant operations
# ============================================================

# Build the small graph used to explain concepts
eoh = 4
g = util.build_website_graph(nnodes=4, rate=3, extra_arc_fraction=0.25, seed=42)
flows, paths = util.build_random_paths(g, min_paths=3, max_paths=5,
                                          min_units=1, max_units=10, eoh=eoh,
                                          seed=42, max_wait=2)
tug = util.build_time_unfolded_graph(g, eoh=eoh)
node_counts, arc_counts = util.get_counts(tug, flows, paths)

# Constraints in the Subproblem

When troubles spring up like mushrooms

## User Habits

**What if we know something about the habits of our users?**

* E.g. we may know that they don't tend to spend a long time on a single page
* We could use this information to _further reguralize the problem_

**Specifically, we could add _a constraint in the subproblem_**

I.e. by putting _a limit on consecutive visits_ to the same node

* It seems simple enough, but in practice it's serious issue

> **Why is that the case?**

* Such a constraint violates a basic assumption in Dijkstra's method
* I.e. that all path information can be condensed into it's length

**With the new constraint, our shortest path method no longer works**

## Walking the Line

**With our shortest path approach, we were walking a fine line**

* The problem could be solved in polynomial time
* ...But even a small addition could make it NP-hard instead

**With the new constraint, pricing becomes indeed NP-hard**

There is nothing we can do about that

* ...But perhaps we can use a better suited technique
* Something designed specifically for NP-hard, combinatorial problems
* ...With lots of messy constraints

**For example, we could use Constraint Programming**

...In its more modern incarnation, _Lazy Clause Generation_ (a.k.a. CP-SAT)

# Constraint Programming and Lazy Clause Generation

Very little is lazy about that

## Constraint Satisfaction Problems

**CP is techniques designed to address Constraint Satisfation Problems (CSPs):**

$$
\langle X, D, C \rangle
$$

Where:

* $X$ is a set of decision variables
* $D$ is the set of their domains
* $C$ is a set of constraints
* $f$ is a cost function

**Almost any decision problem fits those definitions...**

...But in practice, a given CP solver provides

* A library of supported _variables types_
* A library of suported _constraints_

## ...And Constraint Optimization Problems

**CP can handle Constraint Optimization Problems (COP):**

$$
\langle f, X, D, C \rangle
$$

* Where $f$ is a cost function

COPs are tackled as a _sequence of CSPs_ via this scheme:

* $\text{best solution $x^* = \bot$}$
* $\text{while true}$
   $\text{find a solution for $\langle X, D, C \rangle$}$
  - $\text{If a solution $x^\prime$}$ is found:
    - $\text{$x^* = x^\prime$}$
    - $\text{$C = C \cup \{f(x) < f(x^\prime)\}$}$ # We as for an improving solution
  - $\text{otherwise, break the loop}$

The solver state is _maintained between solutions_ so as not to waste effort

## Variables and Constraints

**In terms of supported variables types**

* All CP solver provide _integer_ variables
* Some also provide _numeric_, _interval_, _set_, or _graph_ variables

**In terms of supported constraints**

* All CP solvers provide _equalities_, _inequalities_, _ over _linear expressions_
  - $y = a^T {\bf x}$, $y \leq a^T {\bf x}$
* All CP solvers provide $\neq$ constraints
  - $y \neq x$
* Most CP solvers provide $\max$ and $\min$ constraints
  - $y = \max({\bf x})$, $y = \min({\bf x})$
* Some CP solvers provide products and modulo constraints (over scalars)
  - $y = xy$, $y = x \mod a$


## Variables and Constraints

**CP solver provide also _constraints with non-mathematical nature_**

* E.g. logical constraints:

$$
x \vee y, x \wedge y, x \Rightarrow y \ldots
$$

* E.g. a set of variables should take all different values:

$$
{\scriptsize \text{ALLDIFFERENT}}(x)
$$

* E.g. a set of variables should take/not take values from a table $T$:

$$
{\scriptsize \text{ALLOWED}}(x, T) \quad \text{and} \quad {\scriptsize \text{FORBIDDEN}}(x, T)
$$

* E.g. a set of activities with start times $x$ and durations $d$ should not overlap:

$$
{\scriptsize \text{NOOVERLAP}}(x, d)
$$

## Propagators

**CP solvers are _search based_**

They maintain information about the variable domains in a _Domain Store_:

* The solver may store the domain bounds, i.e. $x_i \in \{lb_i, ... ub_i\}$
* ...Or the individual allowed values, i.e. $x_i \in \{v_0, v_1, \ldots \}$
* Other representations are also possible

**Constraints are associated to algorithms called _propagators_**

* A propagator takes as input the current variable domains
* ...And can _prune_ (some) provably infeasible values

By doing so, we can dramatically reduce the size of the search space

**Propagators often rely on _structural patterns_ to improve pruning**

* E.g. ${\scriptsize \text{ALLDIFFERENT}}(x)$ can prune more than $x_i \neq x_j, \forall i \neq j$
* ...Since it can reason on multiple variables at the same time

## Propagators

**Let's see an example for ${\scriptsize \text{ALLOWED}}([x_0, x_1], T)$, with T given by:**

|  x_0  |  x_1  |
|:-----:|:-----:|
|   0   |   0   |
|   0   |   1   |
|   1   |   1   |

Let's that initially $D_0 = \{0, 1\}$, and $D_1 = \{0, 1\}$

* If $x_0$ looses the value $0$
  - ...Then the ${\scriptsize \text{ALLOWED}}$ propagator prunes $0$ from $D_1$
  - ...Because it no longer has a feasible _support_ in $D_0$
* If $x_1$ looses the value $1$
  - ...Then the ${\scriptsize \text{ALLOWED}}$ propagator prunes $1$ from $D_0$
  - ...Because it no longer has a feasible _support_ in $D_1$

## Propagators and Lazy Clause Generation

In Lazy Clause Generation solvers, propagators have two additional tasks:

**1) Whenever they prune a domain, they also _generate boolean literals_**

* These correspond to the pruning operations
  - E.g. in our two example we would generate literals $[x_0 \neq 0]$ and $[x_1 \neq 1]$
* These literals represent _variables_ associated to the state of a constraint
  - E.g. $[x_0 \neq 0] = 1$ if $0 \notin D_0$ and $[x_0 \neq 0] = 0$ otherwise

**2) Whenever they prune a domain, they also generate an _explanation_**

* This is a logical _clause_ representing the reasoning that led to pruning
  - E.g. in our first example we would generate $[x_1 \neq 0] \Rightarrow [x_1 \neq 0]$
* These clauses are constraints on the literal variables
  - They function like normal constraints (except they are specifically tracked)

## Constraint Propagation

**Pruning can trigger the activation of other propagators**

...In a process called _Constraint Propagation_

* After some activations, the process reaches a _fix point_
* If propagation causes a domain to become empty, we have a _conflict_
* ...In which case we need to backtrack

**As a consequence, propagators are called many times per search node**

We can have millions of propagator calls in a solution process

* For this reason, they often run in _constant or low-degree polynomial_ time
* ...And propagator are heavily optimized
  - E.g. the $\scriptsize \text{ALLOWED}$ constraint is so efficient
  - ...That tables with > 100,000 entry can be handled almost instantly
  - This is achieved by relying on incremental computation

## Constraint Propagation and Implication Graphs

**In LCG solvers, constraint propagation generates an _implication graph_**

This consists of the _literals_, connected by the generated _explanations_

<center style="font-size:small">
<img src="assets/implication_graph.png" width=950px />
Image from <a href="http://satassociation.org/articles/FAIA185-0131.pdf">this book chaper</a>
</center>

* In the pictures, $x$ variables correspond to boolean literals (i.e. constraint states)
* ...And each $\omega$ is an explanation

## Conflict Driven Clause Learning

**In LCG solvers, each search decision also generates a literal**

E.g. if we assign $1$ to $x_0$, we generate $[x_0 = 1]$

<center style="font-size:small">
<img src="assets/implication_graph.png" width=950px />
Image from <a href="http://satassociation.org/articles/FAIA185-0131.pdf">this book chaper</a>
</center>

Each decisions is associated to a _decision value_

* In the picture, they are the number after the @ symbol
* When we make a new decision, we increment the current decision value

## Conflict Driven Clause Learning

**In LCG solvers, each search decision also generates a literal**

E.g. if we assign $1$ to $x_0$, we generate $[x_0 = 1]$

<center style="font-size:small">
<img src="assets/implication_graph.png" width=950px />
Image from <a href="http://satassociation.org/articles/FAIA185-0131.pdf">this book chaper</a>
</center>

Literals generated by propagation are also labeled with a decision value

* ...But in this case there is _no increment_
* In the picture, many literals are associated to decision level 5

## Conflict Driven Clause Learning

**In case of a conflict ($\kappa$ in the figure), an LCG solver can _learn a constraint_**

This technique is referred to as _Conflict Driven Clause Learning_

<center style="font-size:small">
<img src="assets/implication_graph.png" width=950px />
Image from <a href="http://satassociation.org/articles/FAIA185-0131.pdf">this book chaper</a>
</center>

The idea is to identify _which decisions (literals) are to blame_ for the conflict

* First, we identify all literal with the _same decision value_ as the conflict
* The earliest one ($x_1 = 0@5$ in the figure) always corresponds to a decision

## Conflict Driven Clause Learning

**In case of a conflict ($\kappa$ in the figure), an LCG solver can _learn a constraint_**

This technique is referred to as _Conflict Driven Clause Learning_

<center style="font-size:small">
<img src="assets/implication_graph.png" width=950px />
Image from <a href="http://satassociation.org/articles/FAIA185-0131.pdf">this book chaper</a>
</center>

Without this literal, the conflict would not arise

* ...Therefore it will appear in the learned constraint

## Conflict Driven Clause Learning

**In case of a conflict ($\kappa$ in the figure), an LCG solver can _learn a constraint_**

This technique is referred to as _Conflict Driven Clause Learning_

<center style="font-size:small">
<img src="assets/implication_graph.png" width=950px />
Image from <a href="http://satassociation.org/articles/FAIA185-0131.pdf">this book chaper</a>
</center>

To this, we add all literals with decision level _lower_ than the current one

* ...That are connected via explanation to literals in the current decision level
* In the figure, those would be $x_{31} = 0@3$ and $x_{21} = 0@2$

## Conflict Driven Clause Learning

**In case of a conflict ($\kappa$ in the figure), an LCG solver can _learn a constraint_**

This technique is referred to as _Conflict Driven Clause Learning_

<center style="font-size:small">
<img src="assets/implication_graph.png" width=950px />
Image from <a href="http://satassociation.org/articles/FAIA185-0131.pdf">this book chaper</a>
</center>

If we want to avoid the conflict, _at least one_ of these literals should be _false_

Therefore the clause we learn is: $\neg [x_1 = 0] \vee \neg [x_{31} = 0] \vee \neg [x_{21} = 0]$

## Conflict Driven Clause Learning

**The clause we learn is _globally valid_**

...So that we can restart search and we will not make the same mistake again

<center style="font-size:small">
<img src="assets/implication_graph.png" width=950px />
Image from <a href="http://satassociation.org/articles/FAIA185-0131.pdf">this book chaper</a>
</center>

**In other words, we have a _complete_ method that does _not_ rely on tree search**

* CDCL was invented for pure SAT solvers, and it is key to their efficiency
* In LCG we used it to obtain similarly strong benefits

## Some Considerations

**As usual, we have just scratched the surface for CP/LCG**

* You can find more information about classical CP [in this handbook](https://www.elsevier.com/books/handbook-of-constraint-programming/rossi/978-0-444-52726-4)
* ...And for LCG the best starting point are the papers by [Peter Stuckey](https://dblp.org/pid/s/PeterJStuckey.html)

**Unlike MILP, CP does not rely on numerical optimization**

* Combinatorial constraints are first-class citizens
  - E.g. we have no big-Ms here!
* It tends to work best for problems with many combinatorial elements

**Unlike MILP, CP lack a global bounding method**

* There is no LP relaxation, and propagation works at a _local_ level
* CDCL goes a long way towards countering this issue
* ...But sometimes the lack of a global bound leads to weaker performance